In [32]:
from transformers import pipeline

unmasker = pipeline("fill-mask")

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
results = unmasker("This course will teach you all about <mask> models.", top_k=2)
print(results)

In [ ]:
345/1500

## Transformer架构
* Whisper
* ChatGLM3
* SDXL
## Moviepy

In [ ]:
import whisper
model = whisper.load_model("medium") #tiny, base, small, medium, large

In [ ]:
songPath = "翩翩"
song = songPath+"/歌.mp3"
lyrics = songPath+"/歌词.txt" #根据segments调整
timedlyrics = songPath+"/时序歌词.srt"
result = model.transcribe(song,language='Chinese')

## 为了方便与原歌词(歌词.txt)对比, 然后合并歌词, 与原歌词长度对应

In [ ]:
def merge_segments(segments, i):
    i=i-1
    segments[i]['text'] += segments[i+1]['text']
    del segments[i+1]
    for i in range(0,len(segments)-1):
        segments[i]['end'] = segments[i+1]['start']
    for i in range(0,len(segments)-1):
        segments[i]['id'] = i+1    
 

segments = [{'id': item['id']+1, 'start': item['start'], 'end': item['end'], 'text': item['text']} for item in result['segments']]


In [ ]:
merge_segments(segments,15)
segments[:]

## 根据自动识别的歌词分段(text),手动调整歌词与text对应, 然后读入

In [ ]:
lsegments = []

with open(lyrics, 'r') as file:
    for line in file.readlines():
        lsegments.append(line.strip())
lsegments



In [ ]:
lyrics

## text1为调整后的歌词,与自动识别的歌词(text)对应

In [ ]:
for i in range(len(segments)):
    segments[i]['text1'] = lsegments[i]

In [ ]:
segments[:]

In [ ]:
with open(timedlyrics, "w") as file:
    for i, segment in enumerate(segments):
        start_time = segment['start']
        end_time = segment['end']
        text = segment['text1']
        
        file.write(f"{i+1}\n")
        file.write(f"{start_time} --> {end_time}\n")
        file.write(f"{text}\n\n")

## 根据歌词text1生成图形提示词text2

In [ ]:
import requests
import json
import base64
from PIL import Image
import io
import os

llm_url = 'http://0.0.0.0:8000' #本地LLM API地址

def prompt_generation(text1):
    # 发送HTTP POST请求
    
    data = {'prompt':'根据下面的内容描述，生成一副生动的画面并只用英文表示：'+text1}
    json_data = json.dumps(data)

    # 发送HTTP POST请求
    response = requests.post(llm_url, data=json_data, headers={'Content-Type': 'application/json'})
    result_json = json.loads(response.text)
 
    # 输出结果
    if 'response' in result_json:
        return result_json['response']
    else:
        print("Unexpected result:", result_json)
        return None

for i in range(len(segments)):
    segments[i]['text2'] = prompt_generation(segments[i]['text1'])

In [ ]:
segments[-3:]

## 根据text2生成图片到images目录下
novel_dict对sdxl要从新调整

In [ ]:
sd_url = "http://127.0.0.1:7860/sdapi/v1/txt2img"
prompt = "best quality,masterpiece,illustration, an extremely delicate and beautiful,extremely detailed,CG,unity,8k wallpaper, "
negative ="NSFW,sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, bad anatomy,(long hair:1.4),DeepNegative,(fat:1.2),facing away, looking away,tilted head, {Multiple people}, lowres,bad anatomy,bad hands, text, error, missing fingers,extra digit, fewer digits, cropped, worstquality, low quality, normal quality,jpegartifacts,signature, watermark, username,blurry,bad feet,cropped,poorly drawn hands,poorly drawn face,mutation,deformed,worst quality,low quality,normal quality,jpeg artifacts,signature,watermark,extra fingers,fewer digits,extra limbs,extra arms,extra legs,malformed limbs,fused fingers,too many fingers,long neck,cross-eyed,mutated hands,polar lowres,bad body,bad proportions,gross proportions,text,error,missing fingers,missing arms,missing legs,extra digit, extra arms, extra leg, extra foot,"

novel_dict = { #SD1.5
          "enable_hr": "false",
          "denoising_strength": 0,
          "firstphase_width": 0,
          "firstphase_height": 0,
          "hr_scale": 2,
          "hr_upscaler": "string",
          "hr_second_pass_steps": 0,
          "hr_resize_x": 0,
          "hr_resize_y": 0,
          "prompt": "",
          "styles": ["string"],
          "seed": -1,
          "subseed": -1,
          "subseed_strength": 0,
          "seed_resize_from_h": -1,
          "seed_resize_from_w": -1,
          "sampler_name": "DPM++ SDE Karras",
          "batch_size": 1,
          "n_iter": 1,
          "steps": 25,
          "cfg_scale": 7,
          "width":768,#1024
          "height": 1024,#768
          "restore_faces": "false",
          "tiling": "false",
          "do_not_save_samples": "false",
          "do_not_save_grid": "false",
          "negative_prompt": negative,
          "eta": 0,
          "s_churn": 0,
          "s_tmax": 0,
          "s_tmin": 0,
          "s_noise": 1,
          "override_settings": {},
          "override_settings_restore_afterwards": "true",
          "script_args": [],
          "sampler_index": "DPM++ SDE Karras",
          "script_name": "",
          "send_images": "true",
          "save_images": "true",
          "alwayson_scripts": {}
        }
novel_dict2 = { #SDXL: 
    "text_prompt": "",
    "negative_prompt": negative,
    "width": 1344, #最大:1048576, 缺省:512
    "height": 768,
    "steps": 30,
    "cfg_scale": 7,
    "engine":"stable-diffusion-xl-1024-v1-0", #缺省:stable-diffusion-xl-1024-v0-9
    "sampler": "UiPC",#缺省: k_dpmpp_2m
    #"batch_size": 1,
    #"n_iter": 1,
        
    "send_images": "true",
    "save_images": "true",
}
for i in range(1,len(segments)):
    #novel_dict1['prompt']=prompt+segments[i]['text2']
    novel_dict2['text_prompt']=prompt+segments[i]['text2']
    
    html = requests.post(sd_url, data=json.dumps(novel_dict2))
    image_response = json.loads(html.text)
    image_bytes = base64.b64decode(image_response['images'][0])
    image = Image.open(io.BytesIO(image_bytes))
    image_file = songPath+'/images/'+str(segments[i]['id'])+'.jpg'
    image.save(image_file)
    segments[i]['image_file'] = image_file

In [ ]:
import random
import cv2
import numpy as np

txt_speed = 4
h=1024
w=1024



def fl_up(gf, t):
    start = max(0, int(txt_speed*t))
    end = min(int(txt_speed*t)+h, gf(t).shape[0])
    return gf(t)[start:end, :]

def fl_down(gf, t):
    start = max(0, int(txt_speed*t)-h)
    end = min(int(txt_speed*t), gf(t).shape[0])
    return gf(t)[start:end, :]

def fl_left(gf, t):
    start = max(0, int(txt_speed*t))
    end = min(int(txt_speed*t)+w, gf(t).shape[1])
    return gf(t)[:, start:end]

def fl_right(gf, t):
    start = max(0, int(txt_speed*t)-w)
    end = min(int(txt_speed*t), gf(t).shape[1])
    return gf(t)[:, start:end]




def fl_zoom(gf, t, start_scale=1.0, end_scale=1.2):
    image = gf(t)
    if image.size == 0:
        print(f"Empty image at time {t}")
        return image
    scale = start_scale + (end_scale - start_scale) * t / duration
    new_size = (int(image.shape[1] * scale), int(image.shape[0] * scale))
    return cv2.resize(image, new_size)

def fl_shrink(gf, t, start_scale=1.2, end_scale=1.0):
    image = gf(t)
    if image.size == 0:
        print(f"Empty image at time {t}")
        return image
    scale = start_scale + (end_scale - start_scale) * t / duration
    new_size = (int(image.shape[1] * scale), int(image.shape[0] * scale))
    return cv2.resize(image, new_size)

def center_zoom(gf, t, start_scale=1.0, end_scale=1.2):
    image = gf(t)
    if image.size == 0:
        print(f"Empty image at time {t}")
        return image
    
    scale = start_scale + (end_scale - start_scale) * t / duration
    new_size = (int(image.shape[1] * scale), int(image.shape[0] * scale))
    
    # 计算平移量
    dx = (image.shape[1] - new_size[0]) // 2
    dy = (image.shape[0] - new_size[1]) // 2
    
    # 缩放图像
    resized_image = cv2.resize(image, new_size)
    
    # 创建平移矩阵
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    
    # 平移图像
    zoomed_image = cv2.warpAffine(resized_image, M, (image.shape[1], image.shape[0]))
    
    return zoomed_image





In [ ]:
segments[0:4]

In [ ]:
from moviepy.editor import ImageClip, AudioFileClip, TextClip,VideoFileClip, concatenate_videoclips,CompositeVideoClip
from moviepy.video import fx
from moviepy.editor import vfx

font = "/System/Library/Fonts/PingFang.ttc" 
clips = []
# 定义动画函数列表
fl_functions = [fl_up, fl_down, fl_left, fl_right, fl_zoom, fl_shrink]

for ii in range(len(segments)):
    image_file = segments[ii]['image_file']
    duration = segments[ii]['end']-segments[ii]['start']
    text1 = segments[ii]['text1']
    fl = random.choice(fl_functions)
    print(ii, image_file, duration,fl, text1)

    image_clip = ImageClip(image_file)
    image_clip = image_clip.set_duration(duration)  # 设置时长
    

    image_clip = image_clip.fl(fl_functions[i%6])
  

    text_clip = TextClip(text1,fontsize=54,font=font,color='yellow', bg_color='black')
    #text_clip = text_clip.set_position(('center','bottom'))
    text_clip = text_clip.set_position(('center',0.8), relative=True)
    text_clip = text_clip.set_duration(duration)
    text_clip = text_clip.set_opacity(0.6)
    
    composite_clip = CompositeVideoClip([image_clip, text_clip])
    clips.append(composite_clip)
final_clip = concatenate_videoclips(clips)
audio_clip = AudioFileClip(song)
final_clip = final_clip.set_audio(audio_clip)
final_clip.write_videofile(songPath+"/歌.mp4", fps=24, audio_codec="aac")




## 根据歌词片段,生成视频查询关键词

In [ ]:
import requests
import json
import base64
from PIL import Image
import io
import os

llm_url = 'http://0.0.0.0:8000' #本地LLM API地址

sys='''system_prompt:
  # Instructions

  You're a video research expert. The user will give you the timed captions of a video they will make, and you will give back a list of couples of text search queries that will be used to search for background video footage, and the time t1 and t2 when it will be shown in the video.
  # Output format
  The format will be JSON parasable and look like this:
  [[[0.0, 4.4], ["Dog barking", "Dog angry", Canine angry"]], [[4.4, 7.8], "bone", "pet food", "food", "canine"], ect...

  # Time periods t1 and t2
  Time periods t1 and t2 must always be consecutive, and last between 4 to 5 seconds, and must cover the whole video.
  For example, 	[0, 2.5] <= IS BAD, because 2.5-0 = 2.5 < 3
              [0, 11]  <= IS BAD, because 11sec > 5 sec
              [0, 4.2] <= IS GOOD
              
  # Query search string list
  YOU ALWAYS USE ENGLISH IN YOUR TEXT QUERIES
  As you have seen above, for each time period you will be tasked to generate 3 strings that will be searched on the video search engine, to find the appropriate clip to find.
  Each string has to be between ONE to TWO words.
  Each search string must DEPICT something visual.
  The depictions have to be extremely visually concrete, like `coffee beans`, or `dog running`.
  'confused feelings' 			<= BAD, because it doesn't depict something visually
  'heartbroken man' 			<= GOOD, because it depicts something visual. 
  'man feeling very lonely' 	<= BAD, because it contains 4 words.
  The list must always contain 3 query searches.
  ['Sad person']					<= BAD, because it's one string
  ['Sad person', 'depressed man', 'depressed person'] <= GOOD, because it's 3 strings
  ['Une Pomme', 'un enfant qui rit', 'une personne heureuse'] <= BAD, because the text query is NOT in english
'''

def words_generation(text1):
    # 发送HTTP POST请求
    
    #data = {'system_prompt':sys,  'chat_prompt': "Timed captions:"+text1+"Video search queries"}
    data = {'prompt':sys+'chat_prompt: Timed captions:'+text1+'Video search queries'}
    
    json_data = json.dumps(data)

    # 发送HTTP POST请求
    response = requests.post(llm_url, data=json_data, headers={'Content-Type': 'application/json'})
    result_json = json.loads(response.text)
 
    # 输出结果
    if 'response' in result_json:
        return result_json['response']
    else:
        print("Unexpected result:", result_json)
        return None

for i in range(len(segments)):
    segments[i]['text3'] = words_generation(segments[i]['text1'])

## 根据歌词片段, 生成视频clip

In [ ]:
import requests
from moviepy.video.io.VideoFileClip import VideoFileClip

def search_videos(query_string, landscape=True):
    url = "https://api.pexels.com/videos/search"
    headers = {
        "Authorization":"RQdKQPXlzy3URM55teyKcU5taQBPE5J9pFlh7WNG4tcQASjAz0Nkn5l6"
    }
    params = {
        "query": query_string,
        "orientation": "landscape" if landscape else "portrait",
        "per_page": 15
    }

    response = requests.get(url, headers=headers, params=params)
    json_data = response.json()

    return json_data


def getBestVideo(query_string, orientation_landscape=True, used_vids=[]):
    vids = search_videos(query_string, orientation_landscape)
    videos = vids['videos']  # Extract the videos list from JSON

    # Filter and extract videos with width and height as 1920x1080 for landscape or 1080x1920 for portrait
    if orientation_landscape:
        filtered_videos = [video for video in videos if video['width'] >= 1920 and video['height'] >= 1080 and video['width']/video['height'] == 16/9]
    else:
        filtered_videos = [video for video in videos if video['width'] >= 1080 and video['height'] >= 1920 and video['height']/video['width'] == 16/9]

    # Sort the filtered videos by duration in ascending order
    sorted_videos = sorted(filtered_videos, key=lambda x: abs(15-int(x['duration'])))

    # Extract the top 3 videos' URLs
    for video in sorted_videos:
        for video_file in video['video_files']:
            if orientation_landscape:
                if video_file['width'] == 1920 and video_file['height'] == 1080:
                    if not (video_file['link'].split('.hd')[0] in used_vids):
                        return video_file['link']
            else:
                if video_file['width'] == 1080 and video_file['height'] == 1920:
                    if not (video_file['link'].split('.hd')[0] in used_vids):
                        return video_file['link']
    print("NO LINKS found for this round of search with query :", query_string)
    return None

In [ ]:
vurl=getBestVideo('sad eyes',1).split('&')[0]
clip = VideoFileClip(vurl)
clip.write_videofile('temp.mp4')

# 转换heic文件为jpg文件

In [ ]:


def convert_heic_to_jpg(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".heic"):
            heic_file = os.path.join(directory, filename)
            jpg_file = os.path.splitext(heic_file)[0] + ".jpg"
            ! magick convert {heic_file} {jpg_file}

# 指定目录路径
directory_path = "/Users/xjq284/Pictures/xjq"

# 调用函数进行转换
convert_heic_to_jpg(directory_path)

In [ ]:
import os
from PIL import Image

def convert_heic_to_jpg(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".heic"):
            heic_file = os.path.join(directory, filename)
            jpg_file = os.path.splitext(heic_file)[0] + ".jpg"
            
        
            
            # 将HEIC图像转换为PIL图像对象
            image = Image.open(heic_file)
            
            # 保存为JPG格式
            image.save(jpg_file, "JPEG")
            
            # 删除原始的HEIC文件
            os.remove(heic_file)

# 指定目录路径
directory_path = "/Users/xjq284/Pictures/xjq"

# 调用函数进行转换
convert_heic_to_jpg(directory_path)

# 录音5分钟

In [ ]:
import os
import sounddevice as sd
from scipy.io import wavfile
from pydub import AudioSegment

def convert_wav_to_mp3(wav_file, mp3_file):
    # 读取.wav文件
    audio = AudioSegment.from_wav(wav_file)

    # 将音频导出为MP3格式
    audio.export(mp3_file, format="mp3")

def record_audio(duration, filename):
    # 设置录音参数
    sample_rate = 44100  # 采样率
    channels = 1  # 声道数

    # 开始录音
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=channels)

    # 等待录音完成
    sd.wait()

    # 保存录音为.wav文件
    wavfile.write(filename, sample_rate, recording)

# 设置录音时长和保存文件名
duration = 5*60  # 录音时长（秒）
filename = "recording"  # 保存文件名

wav_file = "/Users/xjq284/myvoice/"+filename+".wav"
mp3_file = "/Users/xjq284/myvoice/"+filename+".mp3"

# 调用录音函数
record_audio(duration, wav_file)
convert_wav_to_mp3(wav_file, mp3_file)
os.remove(wav_file)


In [ ]:
from pydub import AudioSegment

def convert_wav_to_mp3(wav_file, mp3_file):
    # 读取.wav文件
    audio = AudioSegment.from_wav(wav_file)

    # 将音频导出为MP3格式
    audio.export(mp3_file, format="mp3")

# 指定输入的.wav文件和输出的MP3文件
wav_file = "recording2.wav"
mp3_file = "recording2.mp3"

# 调用函数进行转换


从一个"
0:06
关于权重和神经网络
0:08
权重是神经网络的参数
0:10
网络的参数"格式文件读入, 然后以以下格式"
1
0:06-->0:08
关于权重和神经网络
2
0:08-->0:10
权重是神经网络的参数
3
0:10-->0:12
网络的参数"
写入另外一个文件

# 把youtube类型字幕转换为srt字母

In [ ]:
from datetime import datetime

def format_time(time_str):
    time_format = "%M:%S"
    time_obj = datetime.strptime(time_str, time_format)
    return time_obj.strftime("%H:%M:%S,%f")[:-3]

def convert_to_srt(input_file, output_file):
    with open(input_file, 'r') as file:
        content = file.read().strip().split('\n')

    srt = ''

    for i in range(0, len(content)-2, 2):
        start_time = format_time(content[i])
        end_time = format_time(content[i+2])
        if len(content[i+1]) > 1:
            text = content[i+1]
        else:
            text = ""
        srt += f"{i//2+1}\n{start_time} --> {end_time}\n{text}\n\n"

    with open(output_file, 'w') as file:
        file.write(srt)

# 指定输入文件和输出文件
input_file = "1.srt"
output_file = "2.srt"

# 调用函数进行转换
convert_to_srt(input_file, output_file)
